<a href="https://colab.research.google.com/github/sergioaugusto94/Classificador_Paginas_Conteudo/blob/main/Desafio_nzn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Desafio NZN

 - Objetivo do desafio:
 
Elaboração de um algoritmo que é capaz de classificar o tipo de um conteúdo publicado em uma página web, seja ele uma *Hard News*, na qual o interesse dos leitores pelo conteúdo  ocorre em um curto período de tempo; ou *Evergreen*, um tipo de notícia perene, que desperta a curiosidade do público mesmo depois de meses da publicação do artigo.
 
Numa companhia como a NZN, na qual a produção de conteúdo é o carro-chefe, saber em qual grupo as publicações se encaixam é de suma importância para a definição das estratégias que serão adotadas em cada grupo, de forma a maximizar a monetização do negócio.

 # Tratamento e análise de dados

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime

Carregamento das bases de dados.

In [ ]:
base = pd.read_excel('dados_cientista_de_dados_envio.xlsx', 
                     sheet_name = 'dbo.caract_treino')
df_view = pd.read_excel('dados_cientista_de_dados_envio.xlsx', 
                     sheet_name = 'dbo.materias_views_treino')


 Conversão das variáveis da coluna que contem a data da publicação da notícia de string para datetime.

In [ ]:
for i in range(base.shape[0]):
    base.iloc[i, 1] = base['data_pub'][i][:10]
base['data_pub'] = pd.to_datetime(base['data_pub'])
 
base

,codmateria,data_pub,titulo,autor,tag,publieditorial,possuilinkafiliacao,tipo
0,116360,2020-11-06,Entenda o que é ransomware: o malware que sequ...,547,Segurança,0,0,Hard News
1,124744,2020-11-25,Cashback: o que é e como funciona esse método ...,632,Mercado,0,0,Hard News
2,129678,2020-11-04,Associação consegue liminar que restringe aume...,523,Mercado,0,0,Hard News
3,134805,2020-11-06,A história da urna eletrônica e das eleições n...,128,Mercado,0,0,Hard News
4,137827,2021-01-19,Não gosta e quer passar longe? Aprenda como ev...,759,Software,0,0,Hard News
...,...,...,...,...,...,...,...,...
276,215853,2021-04-24,Aprenda como usar o BeFunky: editor de fotos p...,1127,Software,0,0,Evergreen
277,215896,2021-04-21,Guia completo de como usar o Google Maps,867,Internet,0,0,Evergreen
278,215923,2021-04-21,Como jogar League of Legends: Wild Rift no cel...,805,Voxel,0,0,Evergreen
279,216006,2021-04-25,Como usar o Fleet do Twitter? Recurso é pareci...,1182,Redes Sociais,0,0,Evergreen


Para melhorar a visualização da base de dados contendo as visitas em cada artigo, foi criado um novo dataframe, na qual as colunas são os dias em que os acessos foram monitorados, os índices são os códigos dos artigos e os elementos dessa matriz são o número de acessos na página web.

In [ ]:
view_data = df_view.pivot_table('audiencia', ['data_audiencia'], 'codmateria' )

 Foi observado que antes da data da publicação dos artigos, existiam acessos à página do artigo, indicando uma possível revisão do artigo. Por isso, esses acessos antes da data da publicação foram removidos, para que não haja nenhuma interferência no algoritmo classificador.

In [ ]:
# Retirando as views antes da data da publicação
for i in view_data.columns:
    view_data[i] = view_data[i].loc[view_data.index >=
                                    base['data_pub'].loc[base['codmateria'] == i].values[0]].fillna(0)
view_data

codmateria,116360,124744,129678,134805,137827,138347,204581,204582,204583,204584,204587,204589,204593,204594,204597,204598,204600,204601,204602,204604,204607,204610,204611,204614,204619,204622,204625,204631,204633,204636,204638,204639,204640,204641,204642,204643,204644,204645,204646,204647,...,214869,214872,214901,214909,214913,214995,214998,215005,215082,215091,215116,215121,215129,215130,215160,215165,215166,215208,215212,215214,215217,215231,215235,215245,215342,215417,215532,215562,215565,215577,215588,215808,215818,215828,215840,215853,215896,215923,216006,216127
data_audiencia,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-10-01,NaN,NaN,NaN,NaN,NaN,NaN,1470.0,1793.0,2929.0,19163.0,406.0,15514.0,NaN,NaN,863.0,NaN,530.0,25715.0,NaN,3062.0,672.0,10892.0,1073.0,268.0,179.0,9767.0,267.0,10273.0,961.0,626.0,NaN,113.0,5016.0,10862.0,550.0,1033.0,5830.0,3313.0,1413.0,1144.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-10-02,NaN,NaN,NaN,NaN,NaN,NaN,564.0,657.0,1570.0,23087.0,147.0,11526.0,NaN,NaN,127.0,NaN,152.0,7138.0,NaN,310.0,262.0,20941.0,254.0,148.0,33.0,15916.0,458.0,2988.0,492.0,2216.0,3581.0,59.0,10918.0,1340.0,315.0,3159.0,25175.0,14374.0,1644.0,5713.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-10-03,NaN,NaN,NaN,NaN,NaN,NaN,937.0,143.0,332.0,3683.0,33.0,6397.0,2492.0,3433.0,36.0,1479.0,21.0,1973.0,1927.0,65.0,38.0,1831.0,68.0,59.0,11.0,7764.0,98.0,372.0,182.0,202.0,1625.0,4.0,3148.0,133.0,33.0,479.0,3545.0,4432.0,697.0,11390.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-10-04,NaN,NaN,NaN,NaN,NaN,NaN,181.0,58.0,214.0,2435.0,12.0,7313.0,1034.0,2092.0,30.0,584.0,23.0,902.0,2552.0,27.0,85.0,660.0,37.0,19.0,6.0,3794.0,42.0,224.0,81.0,110.0,368.0,8.0,1153.0,100.0,49.0,125.0,1732.0,1704.0,301.0,8153.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-10-05,NaN,NaN,NaN,NaN,NaN,NaN,52.0,23.0,146.0,1094.0,7.0,2641.0,550.0,1085.0,34.0,217.0,14.0,383.0,1120.0,22.0,131.0,300.0,22.0,8.0,3.0,3468.0,28.0,198.0,29.0,48.0,123.0,6.0,568.0,77.0,35.0,106.0,1025.0,1089.0,162.0,6460.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-15,1.0,32.0,0.0,18.0,2.0,384.0,1.0,0.0,3.0,1.0,0.0,3.0,4.0,0.0,1.0,0.0,1.0,1.0,24.0,0.0,1.0,5.0,0.0,1.0,0.0,93.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,3.0,5.0,...,862.0,48.0,30.0,35.0,8.0,8.0,37.0,11.0,3.0,327.0,20.0,20.0,5.0,7.0,50.0,46.0,155.0,1.0,3.0,9.0,23.0,41.0,5.0,61.0,7.0,7.0,40.0,1.0,0.0,14.0,17.0,27.0,109.0,31.0,18.0,6.0,4.0,0.0,6.0,4.0
2021-08-16,2.0,69.0,1.0,37.0,1.0,407.0,0.0,0.0,2.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,23.0,1.0,2.0,7.0,0.0,3.0,0.0,101.0,1.0,0.0,1.0,2.0,1.0,0.0,1.0,1.0,0.0,2.0,2.0,2.0,2.0,5.0,...,248.0,76.0,57.0,58.0,10.0,8.0,26.0,13.0,2.0,279.0,42.0,65.0,9.0,5.0,70.0,172.0,160.0,1.0,3.0,20.0,25.0,52.0,6.0,75.0,7.0,19.0,39.0,0.0,1.0,16.0,22.0,29.0,101.0,45.0,12.0,1.0,9.0,2.0,11.0,14.0
2021-08-17,3.0,89.0,0.0,46.0,2.0,393.0,1.0,1.0,3.0,1.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,1.0,19.0,1.0,1.0,6.0,1.0,2.0,0.0,64.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,3.0,1.0,4.0,...,84.0,59.0,61.0,69.0,4.0,7.0,38.0,7.0,5.0,334.0,41.

 Uma das primeiras análises que busquei fazer foi construir alguns parâmetros estatísticos, pois acreditava que seria possível distinguir os dois grupos por esses parâmetros, principalmente pelo desvio padrão, já que eu acreditava que materias *Evergreen* teriam um desvio padrão inferior ao das matéria *Hard News*.

In [ ]:
# Criando algumas variáveis estatísticas para a análise
base = pd.merge(base, df_view.groupby('codmateria').mean(), on = 'codmateria')
base = pd.merge(base, df_view.groupby('codmateria').median(), on = 'codmateria')
base = pd.merge(base, df_view.groupby('codmateria').std(), on = 'codmateria')
base = pd.merge(base, df_view.groupby('codmateria').quantile(q = 0.1).iloc[:, 1], on = 'codmateria')
base = pd.merge(base, df_view.groupby('codmateria').quantile(q = 0.9).iloc[:, 1], on = 'codmateria')
base = pd.merge(base, df_view.groupby('codmateria').quantile(q = 0.95).iloc[:, 1], on = 'codmateria')
base = pd.merge(base, df_view.groupby('codmateria').quantile(q = 0.99).iloc[:, 1], on = 'codmateria')
base = pd.merge(base, df_view.groupby('codmateria').max().iloc[:, 1], on = 'codmateria')
base = pd.merge(base, df_view.groupby('codmateria').min().iloc[:, 1], on = 'codmateria')
 
base.columns = ['codmateria', 'data_pub', 'titulo', 'autor', 'tag', 
                'publieditorial','possuilinkafiliacao', 'tipo', 'media', 
                'mediana','std', 'quartil10', 'quartil90', 'quartil95', 
                'quartil99', 'max', 'min']

In [ ]:
fig = px.scatter(base, x = 'media', y = 'std', color = 'tipo', 
                 width = 800, height = 400, range_x = [0, 100], range_y = [0, 1000])
 
fig.show()

In [ ]:
fig = px.scatter(base, x = 'media', y = 'quartil90', color = 'tipo', 
                 width = 800, height = 400, range_x = [0, 400], range_y = [0, 600])
 
fig.show()

No entanto, ao se observar os gráficos acima, não foi possível extrair nenhum padrão muito claro para classificar o tipo das notícias, embora grande parte das notícias *Evergreen* apresentassem menor desvio padrão. 

Tendo isso em mente, tentei normalizar esses parâmetros estatísticos com o valor máximo de visitas a um artigo.

In [ ]:
# Normalizando as variáveis estatisticas com o máximo valor encontrado
base['media_norm'] = base['media']/base['max']
base['mediana_norm'] = base['mediana']/base['max']
base['std_norm'] = base['std']/base['max']
base['quartil10_norm'] = base['quartil10']/base['min']
base['quartil90_norm'] = base['quartil90']/base['max']
base['quartil95_norm'] = base['quartil95']/base['max']
base['quartil99_norm'] = base['quartil99']/base['max']


In [ ]:
fig = px.scatter(base, x = 'media_norm', y = 'std_norm', color = 'tipo', 
                 width = 800, height = 400, range_x = [0, 0.1], range_y = [0, 0.15])

fig.show()

In [ ]:
fig = px.scatter(base, x = 'media_norm', y = 'mediana_norm', color = 'tipo', 
                 width = 800, height = 400, range_x = [0, 0.08], range_y = [0, 0.08])

fig.show()

Mesmo com a normalização, ainda assim é difícil encontrar um padrão com os parâmetros criados. 

Dessa forma, a analise dos dados continuou em busca de algum parâmetro que pudesse interpretar melhor a forma como as notícias são classificadas. 

- Artigo *Hard News*

In [ ]:
artigo = 134805
x_date = [datetime(2020, 11, 1), datetime(2021, 8, 21)]
mean_line = [base.loc[base['codmateria'] == artigo]['media'].values[0]]*len(x_date)

fig = px.scatter(view_data, x = view_data.index, y = artigo, range_x = x_date)
fig.add_trace(go.Scatter(y = mean_line, x = x_date, name = 'Evasão média', 
                         line = dict(dash = 'dash')))
fig.show()


- Artigo *Evergreen*

In [ ]:
artigo = 215840
x_date = [datetime(2021, 4, 19), datetime(2021, 8, 21)]
mean_line = [base.loc[base['codmateria'] == artigo]['media'].values[0]]*len(x_date)

fig = px.scatter(view_data, x = view_data.index, y = artigo, range_x = x_date)
fig.add_trace(go.Scatter(y = mean_line, x = x_date, name = 'Média de Views', 
                         line = dict(dash = 'dash')))
fig.show()

Busquei então comparar o comportamento de visita às páginas dos dois tipos de notícia ao longo do tempo. E junto com o perfil de visitas, plotei alguns parâmetros estatísticos para tentar encontrar alguma diferença entre os dois rótulos. Ao plotar o perfil de visitas com a média de visitas, pude perceber visualmente que a frequência com que as visitas diárias ultrapassam a média de visitas é superior em notícias Hard News.

Criei então um novo dataframe, que conta o número total de dias que o artigo recebeu uma visita; os dias em que o número total de visitas foi abaixo de um parâmetro estatístico e a razão desse último com o primeiro valor.

In [ ]:
visitas_sts = view_data.count(axis = 0) #contagem do número de dias
visitas_sts.columns = ['Total']

cols = base.columns.values[8:17] #strings dos parâmetros estatísticos

for j in range(len(cols)):
    low = []
    for i in view_data.columns.values:
        # z conta o número de dias em que o número de visitas foi inferior ao 
        # parametro estatístico (j).
        z = view_data[i][view_data[i] < 
                         base[cols[j]].loc[base['codmateria'] == i].values[0]].count()
        low.append(z)
               
    low = pd.DataFrame(low, columns = ['cnt_'+cols[j]])
    visitas_sts = pd.concat([visitas_sts, low.set_index(visitas_sts.index)], 
                            axis = 1)
    # o código abaixo cálcula a razão explicada acima
    visitas_sts['razao_'+cols[j]] = visitas_sts['cnt_' + 
                                                cols[j]]/visitas_sts.iloc[:, 0]
visitas_sts = visitas_sts.rename(columns={0: 'cnt_total'})

# união do dataframe que conta as visitas com o dataframe original
base = pd.concat([base, visitas_sts.set_index(base.index)], axis = 1)


In [ ]:
base.columns

Index(['codmateria', 'data_pub', 'titulo', 'autor', 'tag', 'publieditorial',
       'possuilinkafiliacao', 'tipo', 'media', 'mediana', 'std', 'quartil10',
       'quartil90', 'quartil95', 'quartil99', 'max', 'min', 'media_norm',
       'mediana_norm', 'std_norm', 'quartil10_norm', 'quartil90_norm',
       'quartil95_norm', 'quartil99_norm', 'cnt_total', 'cnt_media',
       'razao_media', 'cnt_mediana', 'razao_mediana', 'cnt_std', 'razao_std',
       'cnt_quartil10', 'razao_quartil10', 'cnt_quartil90', 'razao_quartil90',
       'cnt_quartil95', 'razao_quartil95', 'cnt_quartil99', 'razao_quartil99',
       'cnt_max', 'razao_max', 'cnt_min', 'razao_min'],
      dtype='object')

In [ ]:
fig = px.scatter(base, x = 'cnt_media', y = 'razao_media', color = 'tipo')

fig.show()

Após a criação dos novos parâmetros, busquei plotar a combinação de vários desses parâmetros para buscar os melhores que poderiam ser utilizados como inputs no algoritmo de classificação. Ao plotar a razão_media (quantidade de pontos abaixo da media / quantidade total de visitas) em função da quantidade de pontos abaixo da media, percebemos que exite uma clara segmentação entre os rótulos, sendo possível traçar uma reta que classificaria os pontos acima dessa reta como artigos HN e abaixo como Evergreen. Nesse caso, visualmente, classificariamos incorretamente apenas 6 dos 281 registros, chegando a um aproveitamento de quase 98%. 

Outras análises foram realizadas para entender a relação de alguns parâmetros com os rótulos do tipo de notícia, mas a princípio, vamos nos limitar a alguns parâmetros estatísticos que se mostraram bastante importantes no processo de classificação, de forma a deixar o processamento do algoritmo o mais fluido possível. 

In [ ]:
fig = px.histogram(base, x = 'tag', color = 'tipo', height = 400, width = 800)

fig.show()

Vemos acima que alguns assuntos tem mais chances de virarem notícias *Evergreen* como *Software, Redes Sociais, Internet, Produto* enquanto que *Ciência* e *Mobilidade Urbana* são tags que não se mostram perenes. 

In [ ]:
fig = px.histogram(base, x = 'autor', color = 'tipo', height = 400, width = 800)

fig.show()

Os autores também são bons indicativos do tipo de notícia. Vemos que alguns dos autores são especializados em um determinado grupo de notícias. 

In [ ]:
base['dia_semana'] = 0
for i in range(base.shape[0]):
  base.iloc[i,-1] = base['data_pub'][i].dayofweek

In [ ]:
fig = px.histogram(base, x = 'dia_semana', color = 'tipo', height = 400, width = 800)

fig.show()

É possível observar também que a data da publicação do artigo pode ser um bom indicativo do tipo da notícia e essa informação também pode ser usada como uma estratégia da NZN para que uma determinada publicação atinja o objetivo proposto. No caso, observamos que publicações nos finais de semana (dias 5 e 6) tem mais chance de ser *Evergreen*, enquanto que publicações de segunda, quinta e sexta-feira (dias 0, 3 e 4) tendem a ser *Hard News*. 

# Criação do Algoritmo

Como já vimos anteriormente, os parâmetros estatísticos possuem grande potêncial de gerar um algortimo que tenha uma ótima performance. Então irei selecionar apenas os parâmetros que considerei mais importante pela analise que realizei. 

In [ ]:
base.columns

Index(['codmateria', 'data_pub', 'titulo', 'autor', 'tag', 'publieditorial',
       'possuilinkafiliacao', 'tipo', 'media', 'mediana', 'std', 'quartil10',
       'quartil90', 'quartil95', 'quartil99', 'max', 'min', 'media_norm',
       'mediana_norm', 'std_norm', 'quartil10_norm', 'quartil90_norm',
       'quartil95_norm', 'quartil99_norm', 'cnt_total', 'cnt_media',
       'razao_media', 'cnt_mediana', 'razao_mediana', 'cnt_std', 'razao_std',
       'cnt_quartil10', 'razao_quartil10', 'cnt_quartil90', 'razao_quartil90',
       'cnt_quartil95', 'razao_quartil95', 'cnt_quartil99', 'razao_quartil99',
       'cnt_max', 'razao_max', 'cnt_min', 'razao_min', 'dia_semana'],
      dtype='object')

Foram adicionados outros parâmetros nas *colunas_importantes* mas não se mostraram eficazes em melhorar a performance do algoritmo.

In [ ]:
colunas_importantes = ['media', 'std',  'cnt_media', 'razao_media']

inputs = base[colunas_importantes].values
rotulos = base['tipo'].values

Abaixo, foi feita a normalização das 3 primeiras colunas. A normalização da 4 coluna registrou uma queda no desempenho do algoritmo. 

In [ ]:
from sklearn.preprocessing import StandardScaler 

scaler_data = StandardScaler()
inputs[:, 0:3] = scaler_data.fit_transform(inputs[:, 0:3])

Realização do Tuning dos algoritmos, para descobrir os melhores parâmetros de cada algoritmo. Como visualmente conseguimos determinar a maior parte dos rótulos, não é necessário a construção de algoritmos muito complexos. Nesse problema foram utilizados os seguinte algoritmos: *Random Forest*, *KNN*, *SVM*, e *Redes Neurais*. 

**Obs:** Deixei tudo comentado pois não precisamos fazer o tuning mais de uma vez.

In [ ]:
from sklearn.model_selection import GridSearchCV
 #random Search
    # -----Random Forest
from sklearn.ensemble import RandomForestClassifier
# parametros1 = {'criterion': ['entropy'],
#               'n_estimators': [16, 14, 12, 11],
#               'min_samples_split':[3, 7, 9, 5],
#               'min_samples_leaf':[3, 7, 2, 1]}
# grid_search = GridSearchCV(estimator = RandomForestClassifier(), 
#                            param_grid = parametros1)
# grid_search.fit(inputs, rotulos)
# melhor_parametro = grid_search.best_params_
# melhor_resultado = grid_search.best_score_
# print(melhor_parametro)
# print(melhor_resultado) 
## 'criterion': 'entropy', 'min_samples_leaf': 3, 'min_samples_split': 3, 'n_estimators': 11}
## 0.9083333333333334

    #-----KNN
from sklearn.neighbors import KNeighborsClassifier
# parametros1 = {'n_neighbors': [2,3,5,9,10,11,12,13,14],
#               'p': [1,2], 'weights': ['uniform', 'distance'] }
# grid_search = GridSearchCV(estimator = KNeighborsClassifier(), param_grid=parametros1)
# grid_search.fit(inputs, rotulos)
# melhor_parametro = grid_search.best_params_
# melhor_resultado = grid_search.best_score_
# print(melhor_parametro)
# print(melhor_resultado)
## {'n_neighbors': 10, 'p': 1, 'weights': 'uniform'}
## 0.9048872180451127

    #-----SVM
from sklearn.svm import SVC
# parametros1 = {'tol': [0.001,0.0001,0.00001, 0.01],
#               'C': [2.4, 2.0, 1.8, 3.0, 4.0],
#               'kernel': ['rbf', 'linear', 'poly']
#               }
# search_grid = GridSearchCV(estimator=SVC(), param_grid=parametros1)
# search_grid.fit(inputs, rotulos)
# melhor_parametro = search_grid.best_params_
# melhor_resultado = search_grid.best_score_
# print(melhor_parametro) 
# print(melhor_resultado)
## {'C': 2.0, 'kernel': 'poly', 'tol': 0.001}
## 0.9153508771929826

    #----Redes Neurais
from sklearn.neural_network import MLPClassifier

# parametros1 = {'activation': ['relu', 'logistic', 'tanh'],
#               'solver': ['adam', 'sgd'],
#               'batch_size': [10, 56, 75],
#               'hidden_layer_sizes': [(4), (5, 5, 5), (6), (3), (5, 5)], 
#                'max_iter': [1500]} 
# search_grid = GridSearchCV(estimator=MLPClassifier(), param_grid=parametros1)
# search_grid.fit(inputs, rotulos)
# melhor_parametro = search_grid.best_params_
# melhor_resultado = search_grid.best_score_
# print(melhor_parametro) 
# print(melhor_resultado) 
# # {'activation': 'relu', 'batch_size': 56, 'hidden_layer_sizes': 6, 'max_iter': 1500, 'solver': 'adam'}
# # 0.9050125313283208

Observando o código acima extraimos os melhores parâmetros de cada um dos algoritmos e, selecionei os 3 algoritmos que tiveram melhor score para realizar a validação cruzada. 

In [ ]:
from sklearn.model_selection import KFold, cross_val_score

resultados_svm = []
resultados_random_forest = []
resultados_redes = []

svm_cv = SVC(C = 2, kernel = 'poly', tol = 0.001)

random_forest = RandomForestClassifier(criterion= 'entropy', 
                                       min_samples_leaf = 3, 
                                       min_samples_split = 3, n_estimators = 11)

redes = MLPClassifier(activation = 'relu', batch_size = 56, 
                      hidden_layer_sizes = (6), max_iter = 1500, solver = 'adam')

for i in range(20):
    kfold = KFold(n_splits=4, random_state = i, shuffle = True)

    scores = cross_val_score(random_forest, inputs, rotulos, cv=kfold)
    resultados_random_forest.append(scores.mean())

    scores = cross_val_score(svm_cv, inputs, rotulos, cv = kfold)
    resultados_svm.append(scores.mean())

    scores = cross_val_score(redes, inputs, rotulos, cv = kfold)
    resultados_redes.append(scores.mean())

In [ ]:
import statistics as st

print('Média SVM:', st.mean(resultados_svm), ', std SVM: ' , st.stdev(resultados_svm))
print('Média Random Forest:', st.mean(resultados_random_forest), ', std Random Forest: ' , st.stdev(resultados_random_forest))
print('Média Redes Neurais:', st.mean(resultados_redes), ', std Redes Neurais: ' , st.stdev(resultados_redes))

Média SVM: 0.9373365191146882 , std SVM:  0.0058538286290762504
Média Random Forest: 0.9396428571428571 , std Random Forest:  0.007440658167201378
Média Redes Neurais: 0.9252339034205231 , std Redes Neurais:  0.007054464289266257


Podemos ver que a média dos scores do Random Forest é ligeiramente o maior, porém possui um desvio mais alto que o SVM. Dessa forma, vamos adotar o SVM como nosso classificador. 

Abaixo faremos a divisão da base de dados em treinamento e teste.

In [ ]:
from sklearn.model_selection import train_test_split
 
x_data_treinamento, x_data_teste, y_data_treinamento, y_data_teste = train_test_split(inputs, 
                                                                                      rotulos, 
                                                                                      test_size=0.25, 
                                                                                      random_state=0) 

Treinamento do algoritmo

In [ ]:
svm = SVC(C = 2, kernel = 'poly', tol = 0.001)
svm.fit(x_data_treinamento, y_data_treinamento)

SVC(C=2, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='poly',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

Avaliação do desempenho do algoritmo

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

previsoes_svm = svm.predict(x_data_teste)
print('\n SVM', accuracy_score(y_data_teste, previsoes_svm))
prec_svm = confusion_matrix(y_data_teste, previsoes_svm)
print (classification_report(y_data_teste, previsoes_svm))
print(prec_svm)


 SVM 0.9859154929577465
              precision    recall  f1-score   support

   Evergreen       1.00      0.98      0.99        42
   Hard News       0.97      1.00      0.98        29

    accuracy                           0.99        71
   macro avg       0.98      0.99      0.99        71
weighted avg       0.99      0.99      0.99        71

[[41  1]
 [ 0 29]]


Podemos observar que o algoritmo tem um desempenho quase perfeito, errando a classificação de apenas um registro. O classificador acerta 100% dos casos quando este precisa classifica uma notícia como sendo *Evergreen*, enquanto que o precentual quando classifica uma notícia como sendo *HN* é de quase 97% (28/29).

Podemos então fazer o retreino do classificador com toda a base de dados.

In [ ]:
svm = SVC(C = 2, kernel = 'poly', tol = 0.001)
svm.fit(inputs, rotulos)

SVC(C=2, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='poly',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

# Teste do Algoritmo

In [ ]:
base_nova = pd.read_excel('dados_cientista_de_dados_envio.xlsx', 
                     sheet_name = 'dbo.caract_novas')
df_view_nova = pd.read_excel('dados_cientista_de_dados_envio.xlsx', 
                     sheet_name = 'dbo.materias_views_novas')

Criei uma função que faz todo o tratamento que foi detalhado anteriormente. 

In [ ]:
from tratamento_nzn import tratamento

input_nova = tratamento(base_nova, df_view_nova)

Vamos passar a saída da função de tratamento para que ele faça as previsões. 

In [ ]:
previsoes = svm.predict(input_nova)

previsoes = pd.DataFrame(previsoes, columns = ['tipo'])

Com as previsões na mão, vamos juntar-las com os códigos de cada matéria para exportar para csv final. 

In [ ]:
final_df = base_nova['codmateria']

final_df = pd.concat([final_df, previsoes], axis = 1)


Para finalizar, exportamos o dataframe para um arquivo CSV.

In [ ]:
final_df.to_csv(r'/content/resultados_da_classificacao.csv', index = False)